In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# -----------------------
# 1. Prepare the data
# -----------------------
colors = ["green", "fuchsia", "periwinkle", "red", "blue", "purple", "orange", "cyan"]

# Join all colors into one string with a delimiter (to allow transitions between words)
text = " ".join(colors)

# Build character vocabulary
vocab = sorted(set(text))
char2idx = {ch: i for i, ch in enumerate(vocab)}
idx2char = {i: ch for i, ch in enumerate(vocab)}
vocab_size = len(vocab)

# Convert entire text to indices
encoded = np.array([char2idx[c] for c in text])

# Define sequence length (how many characters in input window)
seq_length = 2
inputs, targets = [], []

# Create overlapping sequences — starting from ANY position
for i in range(len(encoded) - seq_length):
    inputs.append(encoded[i:i+seq_length])
    targets.append(encoded[i+seq_length])

inputs = np.array(inputs)
targets = np.array(targets)

# -----------------------
# 2. Define the model
# -----------------------
embedding_dim = 16
rnn_units = 64

model = Sequential([
    Embedding(vocab_size, embedding_dim, batch_input_shape=[None, seq_length]),
    LSTM(rnn_units, return_sequences=False),
    Dense(vocab_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# -----------------------
# 3. Train the model
# -----------------------
model.fit(inputs, targets, batch_size=32, epochs=100)

# -----------------------
# 4. Example: Predict next char
# -----------------------
def predict_next_char(seed):
    # Convert seed string to indices
    encoded_seed = np.array([char2idx[c] for c in seed])
    encoded_seed = encoded_seed[np.newaxis, :]  # add batch dimension
    probs = model.predict(encoded_seed, verbose=0)[0]
    next_idx = np.argmax(probs)
    return idx2char[next_idx]

print()
print("--- BEGIN PREDICTED CHARS (MAY PREDICT SPACES) ---")
print(predict_next_char("re"))   # Could predict 'd' or 'e' depending on learned weights
print(predict_next_char("gr"))   # Likely 'e'
print(predict_next_char("a "))   # p

Epoch 1/100
2/2 [==============================] - 2s 9ms/step - loss: 2.9949
Epoch 2/100
2/2 [==============================] - 0s 8ms/step - loss: 2.9919
Epoch 3/100
2/2 [==============================] - 0s 7ms/step - loss: 2.9892
Epoch 4/100
2/2 [==============================] - 0s 7ms/step - loss: 2.9863
Epoch 5/100
2/2 [==============================] - 0s 7ms/step - loss: 2.9835
Epoch 6/100
2/2 [==============================] - 0s 6ms/step - loss: 2.9809
Epoch 7/100
2/2 [==============================] - 0s 6ms/step - loss: 2.9777
Epoch 8/100
2/2 [==============================] - 0s 12ms/step - loss: 2.9745
Epoch 9/100
2/2 [==============================] - 0s 21ms/step - loss: 2.9714
Epoch 10/100
2/2 [==============================] - 0s 11ms/step - loss: 2.9678
Epoch 11/100
2/2 [==============================] - 0s 12ms/step - loss: 2.9641
Epoch 12/100
2/2 [==============================] - 0s 11ms/step - loss: 2.9598
Epoch 13/100
2/2 [==============================] - 0s 3